In [1]:
import pandas as pd
#from utils import randomSplit, Accumulator
#import torch
#from torch import nn
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # 归一化
from sklearn.model_selection import KFold         # k折交叉验
from sklearn.decomposition import PCA             # pca
from sklearn.decomposition import FactorAnalysis  # 支持因子旋转
import umap
import sklearn
import sklearn.manifold as manifold
import matplotlib.pyplot as plt
#from kmapper.jupyter import display
from sklearn import cluster
import networkx as nx

In [2]:
# 将mapper方法和其他聚类、主成分方法对比
# 1 pca
from factor_analyzer import FactorAnalyzer

class myDataset():
    def __init__(self, renorm=False, pca=False, savehtml=False, **kargs):
        self.player = pd.read_csv('datacleaning/modified_data.csv', header=0, encoding='unicode_escape', delimiter=';')
        #self.player = self.player.drop(['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born'], axis=1)
        
        # Squad 和 Season 需要保留
        #self.player = self.player.drop(['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born'], axis=1)
        
        if renorm:
            scaler = StandardScaler()  # z score 归一化
            #scaler = MinMaxScaler()    # 0-1 归一化
            for col in self.player.columns:
                if col in ('Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born', 'Squad'): continue
                self.player[[col]] = scaler.fit_transform(self.player[[col]])
        
        #project data
        self.X_features_numeric = self.player.drop(['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born', 'Squad'], axis=1).values  # data array
        self.index2col = list(self.player.drop(['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born', 'Squad'], axis=1).columns)  # 将 values 的列id转化为对应的col名称
        self.PlayerPosition = np.array(self.player['Pos'])
        
    def perform_pca1(self, n_components=15):
        # 执行pca并且，做一些可解释性的工作
        fa = FactorAnalysis(n_components=n_components, rotation='varimax')
        # 拟合数据并进行因子分析
        fa_result = fa.fit_transform(self.X_features_numeric)
        print(np.sum(fa.components_ ** 2, axis=1))
        print(np.sum(fa.components_ ** 2))
        # 打印因子载荷矩阵
        print("Factor Loadings: ")
        for i in range(n_components):
            factor_load = fa.components_[i,:]
            big5_ids = np.argsort(np.abs(factor_load))[-1:-7:-1]
            print(f'factor{i+1}: ', end='')
            print(';'.join([f'{self.index2col[id]} {factor_load[id]:.2f}' for id in big5_ids]))
        print(fa.components_.shape)
        # 打印降维后的结果
        print("Factor Analysis Result:")
        print(fa_result.shape)
        return fa_result

    def perform_pca2(self, n_components=10):
        # 执行pca并且，做一些可解释性的工作
        fa = FactorAnalyzer(n_factors=n_components, rotation='varimax', method='principal')
        # 拟合数据并进行因子分析    
        fa_result = fa.fit(self.X_features_numeric)         
        # 得到特征值ev、特征向量v
        ev,v = fa.get_eigenvalues()
        # 同样的数据绘制散点图和折线图
        plt.scatter(range(1, self.X_features_numeric.shape[1] + 1), ev)
        plt.plot(range(1, self.X_features_numeric.shape[1] + 1), ev)
        # 显示图的标题和xy轴的名字
        # 最好使用英文，中文可能乱码
        plt.title("Scree Plot")  
        plt.xlabel("Factors")
        plt.ylabel("Eigenvalue")
        plt.grid()  # 显示网格
        plt.show()  # 显示图形

        print(np.sum(fa.loadings_ ** 2, axis=1))
        print(np.sum(fa.loadings_ ** 2))
        # 打印因子载荷矩阵
        print("Factor Loadings: ")
        for i in range(n_components):
            factor_load = fa.loadings_[i,:]
            big5_ids = np.argsort(np.abs(factor_load))[-5::-1]
            print(f'factor{i}: ', end='')
            print('; '.join([f'{self.index2col[id]}, {factor_load[id]:.3f}' for id in big5_ids]))
        print(fa.loadings_.shape)



    


dataset = myDataset(renorm=1)
dataset.perform_pca1()

[0.18290211 0.04262779 0.01940165 0.36903814 0.02654405 0.12919813
 0.02484422 0.00502491 0.01435711 0.03371653 0.02315728 0.0117518
 0.00806822 0.07985151 0.01630076]
0.9867842293294387
Factor Loadings: 
factor1: PasGround 0.11;PasMedCmp 0.11;PasMedAtt 0.10;PasCmp 0.10;PasTotCmp 0.10;PasTotDist 0.09
factor2: TklWon 0.10;TklW 0.10;Tkl+Int 0.06;TklDri% 0.06;Tkl 0.05;TklMid3rd 0.05
factor3: PasLonCmp% -0.10;PasLonCmp -0.04;PasTotCmp% -0.04;PasTotDist -0.02;PasDead 0.02;PasBlocks 0.02
factor4: Starts -0.30;MP -0.29;Min -0.29;90s -0.29;DriSucc% -0.09;PasLonCmp% -0.05
factor5: PasDead 0.09;TI 0.08;PasLow 0.04;PasCrs 0.04;Crs 0.04;PasHigh 0.03
factor6: TouDefPen -0.15;Rec% -0.10;TouDef3rd -0.10;PasTotPrgDist -0.08;SoT% 0.07;RecProg 0.07
factor7: RecProg -0.07;CarMis -0.04;RecTarg -0.04;CarDis -0.04;TouAtt3rd -0.03;Shots -0.03
factor8: GcaPassLive 0.03;Assists 0.03;PasAss 0.02;GCA 0.02;PasCrs 0.01;Crs 0.01
factor9: PasTotCmp% -0.05;PasLonAtt 0.04;PasHigh 0.04;PasMedCmp% -0.03;PasShoCmp% -0.03

array([[-0.46442351,  0.21478265,  0.73303474, ...,  0.40337644,
        -0.37212495, -0.95030691],
       [ 0.11408015,  0.29448883,  0.67047479, ..., -1.04228316,
        -0.58997633, -0.34004523],
       [ 1.36332997,  0.00973997, -0.33297282, ..., -0.17746566,
         0.33594253, -0.80550374],
       ...,
       [ 0.69871557, -0.03125234, -0.05111861, ...,  0.36858092,
         0.94429613,  0.38976982],
       [-0.93849067, -0.83698202,  0.47285576, ...,  0.90346374,
        -0.19259056, -1.02460554],
       [-0.19614657,  0.06513627,  0.14532682, ..., -0.33279247,
         0.44132573, -0.28990924]])

In [5]:
import plotly.express as px # for data visualization
def chart(X, y):
    #--------------------------------------------------------------------------#
    # This section is not mandatory as its purpose is to sort the data by label
    # so, we can maintain consistent colors for digits across multiple graphs

    # Concatenate X and y arrays
    arr_concat = np.concatenate((X, y.reshape(y.shape[0],1)), axis=1)
    # Create a Pandas dataframe using the above array
    df=pd.DataFrame(arr_concat, columns=['x', 'y', 'z', 'label'])
    # Convert label data type from float to integer
    #df['label'] = df['label'].astype(int)
    # Finally, sort the dataframe by label
    df.sort_values(by='label', axis=0, ascending=True, inplace=True)
    #--------------------------------------------------------------------------#

    # Create a 3D graph
    fig = px.scatter_3d(df, x='x', y='y', z='z', color=df['label'].astype(str), height=700, width=950)

    # Update chart looks
    fig.update_layout(title_text='UMAP',
        showlegend=True,
        legend=dict(orientation="h", yanchor="top", y=0, xanchor="center", x=0.5),
        scene_camera=dict(up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=-0.1),
        eye=dict(x=1.5, y=-1.4, z=0.5)),
        margin=dict(l=0, r=0, b=0, t=0),
        scene = dict(xaxis=dict(backgroundcolor='white',
        color='black',
        gridcolor='#f0f0f0',
        title_font=dict(size=10),
        tickfont=dict(size=10),
    ),
        yaxis=dict(backgroundcolor='white',
        color='black',
        gridcolor='#f0f0f0',
        title_font=dict(size=10),
        tickfont=dict(size=10),
    ),
        zaxis=dict(backgroundcolor='lightgrey',
        color='black',
        gridcolor='#f0f0f0',
        title_font=dict(size=10),
        tickfont=dict(size=10),
    )))
    # Update marker size
    fig.update_traces(marker=dict(size=3, line=dict(color='black', width=0.1)))

    fig.show()

def perform_umap(dataset):
    # umap敏感性分析测试三种情况，直接umap，isomap+umap，pca+umap
    x = dataset.X_features_numeric
    y = dataset.PlayerPosition
    #isomaper = manifold.Isomap(n_components=10, n_jobs=-1)
    isomaper = PCA(n_components=10)  # 这算NTR吧
    umper = umap.UMAP(n_components=3)
    #x = isomaper.fit_transform(x)  # 想知道，有没有什么很好的依据，让我加这个
    x = umper.fit_transform(x)
    chart(x, y)

    
perform_umap(dataset)

# 通过尝试多种参数，发现空洞是不显著的，但是主体部分的三层结构是显著的，后卫的两大分支也是显著的
# 总结一下
# 在显示拓扑结构上，mapper不如既有的umap等方法简洁直观，在发现空洞和flare上以及对其进行量化研究上，
#        mapper存在优势（不过我们的算法有待完善，实际作用有待在空洞更加明显的数据集上做更进一步的检验）